In [1]:
!wget https://raw.githubusercontent.com/opencontext/archaeology-images-ai/main/json_data/artifact_images_w_sentence_captions.json

--2023-10-10 10:33:54--  https://raw.githubusercontent.com/opencontext/archaeology-images-ai/main/json_data/artifact_images_w_sentence_captions.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62284991 (59M) [text/plain]
Saving to: ‘artifact_images_w_sentence_captions.json’

artifact_images_w_s 100%[===================>]  59.40M  6.31MB/s    in 11s     

2023-10-10 10:34:06 (5.25 MB/s) - ‘artifact_images_w_sentence_captions.json’ saved [62284991/62284991]



In [3]:
## this code is just to append better captions to data that has already been downloaded
## with the poorer captions

import json
import os

def process_chunk(artifact_dict):
    for artifact in artifact_dict:
        for train in train_lines:
            # Split the filename and the extension so that we only compare the filename
            if os.path.splitext(train['image'].split('/')[-1])[0] == artifact['media__uuid']:
                train['caption'] = artifact['caption']
    return train_lines

artifact_file = 'artifact_images_w_sentence_captions.json'
train_file = 'train.json'
batchsize = 1000 # Customize this as needed

train_lines = [json.loads(line) for line in open(train_file)]

with open(artifact_file) as f:
    artifact_dict = json.load(f)  # Changed this line to json.load()

    # Process in chunks
    for i in range(0, len(artifact_dict), batchsize):
        artifact_chunk = artifact_dict[i:i+batchsize]
        train_lines = process_chunk(artifact_chunk)

with open(train_file, 'w') as f:
    for item in train_lines:
        f.write("%s\n" % json.dumps(item))

In [11]:
import os
import json
import requests
import pandas as pd
from urllib.parse import urlparse
from urllib.request import urlretrieve
from sklearn.model_selection import train_test_split
import concurrent.futures

# Load JSON from remote URL
url = "https://raw.githubusercontent.com/opencontext/archaeology-images-ai/main/json_data/artifact_images_w_sentence_captions.json"
response = requests.get(url)
data = response.json()

# Randomly select records
df = pd.DataFrame(data)
train_df, rem_df = train_test_split(df, train_size=10, random_state=42)
test_df = rem_df.sample(2, random_state=42)

# Download and rename images
def download_and_rename(row, folder):
    os.makedirs(folder, exist_ok=True)  
    uri = row['image_file__uri']
    uuid = row['media__uuid']
    caption = row['caption']
    parse_object = urlparse(uri)
    _, ext = os.path.splitext(parse_object.path)
    new_image_name = uuid + ext
    new_image_path = os.path.join(folder, new_image_name)
    urlretrieve(uri, new_image_path)
    return {"image": new_image_path, "caption": caption}

# Writing to 'jsonl' files
def write_to_jsonl(new_data, jsonl_file):
    with open(jsonl_file, 'w') as file:
        for json_dict in new_data:
            line = json.dumps(json_dict)
            file.write(line + "\n")

# Process train and test data
with concurrent.futures.ThreadPoolExecutor() as executor:
    train_data = list(executor.map(download_and_rename, [row for _, row in train_df.iterrows()], ['ttrain_images']*len(train_df)))
    test_data = list(executor.map(download_and_rename, [row for _, row in test_df.iterrows()], ['ttest_images']*len(test_df)))

# Write train/test data to jsonl files
write_to_jsonl(train_data, 'ttrain.json')
write_to_jsonl(test_data, 'ttest.json')